## Imports

In [1]:
import sys
import os
import importlib
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import shapiro, kstest
from scipy import stats  

from sklearn.linear_model import LogisticRegression, Ridge, Lasso, ElasticNet, RidgeCV, LassoCV,ElasticNetCV, LogisticRegressionCV
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, accuracy_score, classification_report, f1_score, matthews_corrcoef, mean_squared_error,r2_score, roc_auc_score, roc_curve, auc, confusion_matrix, log_loss
from sklearn.datasets import make_classification
from sklearn.preprocessing import MinMaxScaler
from statsmodels.stats.diagnostic import kstest_normal
from timeit import default_timer as timer
from tqdm import tqdm  
from typing import Optional
from joblib import Parallel, delayed
import pickle

module_path = str(Path("../src/data").resolve())
if module_path not in sys.path:
    sys.path.append(module_path)

import LogRegFxF as LR
import preprocessing as prep

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

In [2]:
sys.path

['/home/lestrada/miniconda3/envs/tumor_type_clasifier/lib/python313.zip',
 '/home/lestrada/miniconda3/envs/tumor_type_clasifier/lib/python3.13',
 '/home/lestrada/miniconda3/envs/tumor_type_clasifier/lib/python3.13/lib-dynload',
 '',
 '/home/lestrada/miniconda3/envs/tumor_type_clasifier/lib/python3.13/site-packages',
 '/home/lestrada/tumor_type_prediction/src/data']

In [8]:
import LogRegFxF as LR
import preprocessing as prep
import feature_selection as fs

In [39]:
importlib.reload(fs)

dir(fs)

['ElasticNet',
 'ElasticNetCV',
 'GridSearchCV',
 'Lasso',
 'LassoCV',
 'LogisticRegression',
 'LogisticRegressionCV',
 'MinMaxScaler',
 'Ridge',
 'RidgeCV',
 'StandardScaler',
 'StratifiedKFold',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'accuracy_score',
 'auc',
 'binary_labeling',
 'chi2',
 'classification_report',
 'confusion_matrix',
 'cross_val_score',
 'f1_score',
 'fdrcorrection',
 'get_high_confidence_proteins',
 'hparameter_grid_search',
 'importlib',
 'kstest',
 'kstest_normal',
 'make_classification',
 'make_scorer',
 'matthews_corrcoef',
 'mean_squared_error',
 'np',
 'pd',
 'precision_recall_curve',
 'r2_score',
 'roc_auc_score',
 'roc_curve',
 'shapiro',
 'timer',
 'train_test_split']

## Data Import

In [5]:
#Proteins quantification intensities file
processed_data = '2024.10.23_CJ_pancancer_250/'
folder_path = '/media/kusterlab/internal_projects/active/TOPAS/WP31/Playground/Retrospective_study/'
PREPROCESSED_FP_INTENSITY = 'preprocessed_fp_with_ref.csv'
intensity_path_file = folder_path + processed_data + PREPROCESSED_FP_INTENSITY
input_quantifications = pd.read_csv(intensity_path_file)

#--------------------------------------------------------------------------------

#Samples metadata (oncotree classification) file.
metadata_path = '/media/kusterlab/internal_projects/active/TOPAS/WP31/Playground/Retrospective_MTBs_Evaluation/'
metadata_file = 'METADATA_PAN_CANCER_Batch300.xlsx'
the_metadata_file = metadata_path + metadata_file
input_metadata = pd.read_excel(the_metadata_file)

#--------------------------------------------------------------------------------

# # Proteins quantification z-scores file
# processed_data = '2024.10.23_CJ_pancancer_250/'
# folder_path = '/media/kusterlab/internal_projects/active/TOPAS/WP31/Playground/Retrospective_study/'
# PREPROCESSED_FP_INTENSITY = 'full_proteome_measures_z.tsv'
# intensity_path_file = folder_path + processed_data + PREPROCESSED_FP_INTENSITY
# df_Z_scores = pd.read_csv(intensity_path_file, sep='\t')

## Data Preprocessing

In [6]:
#Peptides quantification intensities post-processing

# Protein quantification intensities post-processing
input_quantifications = input_quantifications.set_index(input_quantifications.columns[0])
peptides_quant_info = prep.post_process_meta_intensities(input_quantifications.iloc[:,int(input_quantifications.shape[1]/2):].T ) #clean dataframe from regex characers
proteins_quant = input_quantifications.iloc[:,:int(input_quantifications.shape[1]/2)].T #subset protein measurements from dataset

#Imputation
prot_quant_imputed = prep.impute_normal_down_shift_distribution(proteins_quant) #Imputation of missing values in protein intensities using normal distribution down-shift method
na_columns = prot_quant_imputed.isna().any()
na_columns_true = na_columns[na_columns].index.tolist()
print("Proteins with  empty values:", na_columns_true)

#Cleaning sample names
prot_quant_imputed.reset_index(inplace=True)
prot_quant_imputed.rename(columns={'index': 'Sample name'}, inplace=True)
prot_quant_imputed['Sample name'] = prot_quant_imputed['Sample name'].str.replace('pat_', '')

#Dataset with protein intensities and metadata
samples_metadata = input_metadata[["Sample name", "code_oncotree",]] #sample metadata e.g. class, TCC, tissue of origin, etc.
initial_df = samples_metadata.merge(prot_quant_imputed, left_on='Sample name', right_on='Sample name')

#Peptides quantification to binary dataset
peptides_df_binary = pd.DataFrame(
    np.where(peptides_quant_info > 1, 1, 0), #if the # of peptides > 1, then turns to 1, otherwise 0. 
    index=peptides_quant_info.index,
    columns=peptides_quant_info.columns  
)
peptides_df_binary.reset_index(inplace=True) #Moves the index to a column. Allows to obtain patient id
peptides_df_binary.replace('Identification metadata ','',regex=True, inplace=True) #Removes text from id's
peptides_df_binary = samples_metadata.merge(peptides_df_binary, left_on='Sample name', right_on='index') #merging both data sets by Sample Name, ontaining a dataset with sample, classification and peptide binary count
peptides_df_binary.drop('index', axis=1, inplace=True)

peptides_df_binary

(2135, 13017)


/home/lestrada/miniconda3/envs/tumor_type_clasifier/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:2015: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/lestrada/tumor_type_prediction/src/data/preprocessing.py:50: RuntimeWarning: Mean of empty slice
  temp_mean = np.nanmean(temp)


Proteins with  empty values: ['PTGER4', 'CD19', 'FOXO4', 'CRYGA', 'HNRNPCL3;HNRNPCL4', 'MYBPHL']


,Sample name,code_oncotree,MSH6,PCLAF,UTP18,SEC16A,IPO7,EIF3L,RPAP3,INTS3,...,ROPN1L,CARD10,ZNF804A,ZNF503,HHEX,STK40,FAM214A,WNT10B,VMO1,CCDC152
0,H021-3RLVZS-T1-Q1,AASTR,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,H021-VFM3B1-T1-Q1,AASTR,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,H021-3RLVZS-T1-Q1-R2,AASTR,1,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,H021-XBLS3R-M1-Q1,AASTR,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,H021-M2MSRE-M1-Q1,ACBC,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1655,H021-25HCGP-M2-Q1,VMM,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1656,H021-VYS51F-M1-Q1,VSC,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1657,H021-1B7R18-M1-Q1,VSC,1,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1658,H021-FUFZFT-T1-Q1,VSC,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


## Data Set Split

In [10]:
#Removing samples not part of the Oncotree classification
NOS_cases = ['CUPNOS', 'ADNOS', 'SARCNOS', 'SCCNOS', 'missing', 'SOLIDNOS', 'RCSNOS', 'GCTNOS']
ml_initial_df = prep.remove_class(initial_df, NOS_cases, 'code_oncotree')

# Splitting dataset into training and held-out sets1
training_df, held_out_df = prep.data_split(ml_initial_df, split_size=0.25, classified_by='code_oncotree', export=False)


Removed samples: 191
Remaining samples: 1469
Classes with only one sample: 70
Training set samples: 1119
Held-out set samples: 350


## Class Specific Worflow

In [12]:
ARMS_class = ['ARMS'] 
classified_by = 'code_oncotree'
samples_column = 'Sample name'

#Obtaining high confidence proteins by peptides
arms_proteins_by_peptides = fs.get_high_confidence_proteins(peptides_df_binary, ARMS_class, classified_by, threshold=0.7)

# Binary labeling for specific class classification - CREATE A FX or CLASS to do this alltogether with the following code
ARMS_training_df = fs.binary_labeling(training_df, classified_by=classified_by, true_class=ARMS_class)
ARMS_ho_df = fs.binary_labeling(held_out_df, classified_by=classified_by, true_class=ARMS_class)

# 1st Filter - Filtering ARMS training and held-out dataframes by proteins with peptides
ARMS_training_df = ARMS_training_df.filter(items=[samples_column, classified_by, 'Classifier'] + arms_proteins_by_peptides)
ARMS_ho_df = ARMS_ho_df.filter(items=[samples_column, classified_by, 'Classifier'] + arms_proteins_by_peptides)

 6336 proteins identified in 70.0% of ['ARMS'] samples

Number of samples per class:
Classifier
0    1076
1      43
Name: count, dtype: int64


Number of samples per class:
Classifier
0    336
1     14
Name: count, dtype: int64



## Feature Selection

### Hyperparametes for ElasticNet

In [40]:
ARMS_cv_results, ARMS_best_params, ARMS_best_score, ARMS_grid_search_obj = fs.hparameter_grid_search(ARMS_training_df.iloc[:, 0:20], 4, [0.3, 0.5, 0.7], [1, 10], classified_by='code_oncotree')

Grid search completed in 5.50 seconds
Best parameters: {'C': 10, 'l1_ratio': 0.3, 'max_iter': 10000, 'penalty': 'elasticnet', 'solver': 'saga'}
Best score: 0.6691314367649908


### ElasticNet Cross-Validation

In [81]:
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
output_dir = os.path.join(project_root, 'data', 'data_output')
os.makedirs(output_dir, exist_ok=True)

def elnet_cross_val (df:pd.DataFrame, l1_ratio:float, C:float, r=1, classified_by='code_oncotree'):
    """
    This function return the coefficients of the independent variables defined by ML Logistic Regression. Uses 5k Cross Validation. 
    df:input the imputated and concatenated dataframe, generated with previous functions.
    r: random state of CV

    """

    log_reg_coeff_list = []
    results = []

    
    y_train = df['Classifier'] 
    X_train = df.drop(columns=['Sample name', 'Classifier', classified_by], axis=1) 

    #Defining model parameters
    log_reg = LogisticRegression(penalty='elasticnet',  
                                 solver='saga', 
                                 l1_ratio=l1_ratio ,
                                 max_iter=10000, 
                                 C = C, 
                                 class_weight= 'balanced',
                                 warm_start=True,
                                )
    #Saving the id used for each fold:
        
    # Create StratifiedKFold object.
    skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=r)
    p = 1 
    for train_index, test_index in skf.split(X_train, y_train):
        x_train_fold, x_test_fold = X_train.iloc[train_index], X_train.iloc[test_index]
        y_train_fold, y_test_fold = y_train.iloc[train_index], y_train.iloc[test_index]
        log_reg.fit(x_train_fold, y_train_fold)
        
    #Logaritmic regression model coeffiecient:
        log_reg_coeff_list = log_reg.coef_.tolist()
    
    #Logaritmic regression model coeffiecient INTERCEPT:
        log_reg_coeff_list[0].append (log_reg.intercept_[0])
        
    #Model predictions for score in folds
        y_predict_fold = log_reg.predict(x_test_fold)
                
    #Intercept        
        results.append(log_reg_coeff_list[0])
        
    #F1_score for class 1 - entity prediction
        f1_score_class_1 = f1_score(y_test_fold, y_predict_fold, pos_label=1)
        log_reg_coeff_list[0].append(f1_score_class_1)
    
    #F1_score for class 0 - entity prediction
        f1_score_class_0 = f1_score(y_test_fold, y_predict_fold, pos_label=0)
        log_reg_coeff_list[0].append(f1_score_class_0)
    
    #F1_score for class 1 - entity prediction
        f1_score_class_weighted = f1_score(y_test_fold, y_predict_fold, average= 'weighted')
        log_reg_coeff_list[0].append(f1_score_class_weighted)
    
    #MCC Score
        MCC_score = matthews_corrcoef(y_test_fold, y_predict_fold)
        log_reg_coeff_list[0].append(MCC_score)


    col_names = X_train.columns.tolist()
    col_names.extend(['Intercept','F1_1', 'F1_0','F1_weighted','MCC_score' ])
    
    
    coeff_score_df = pd.DataFrame(results)
    coeff_score_df.columns = col_names
    
    return(coeff_score_df)

#--------------------------------------------------------------------

def ml_log_reg_coefficients (df:pd.DataFrame, tumor_type_name:str,l1_ratio:float, C:float, r=1, export=True) -> pd.DataFrame:
    
    """ 
    Wrapper funtion of ml_log_reg_5cv. Executes multiple times a logistic regression, saving the coefficients and scores for each fit of the data. 
    Returns: Returns a dataframe with the coefficients of Logistic Regression
    df: Input dataframe with proteins intensities, imputated and with no NaN values
    r: input the number of repetitions. r = 1 : 1 experiment with 5k Cross validation 

    """
    if not isinstance(r, int) or r <= 0:
        raise ValueError("The number of repetitions 'r' must be a positive integer.")
    if df.empty:
        raise ValueError("Input dataframe 'df' is empty. Please provide a valid dataframe.")

    repetitions = range(r)#number of times the ML algorithm will run. triesx5 = #coefficients


    results = Parallel(n_jobs=8)(
        delayed(elnet_cross_val)(df, l1_ratio, C, iteration) for iteration in tqdm(repetitions, desc="Running Logistic Regression", unit="iteration")
    )

    # Concatenate the results into one DataFrame
    df_concatenated = pd.concat(results, ignore_index=True)

    #Export
    if export:
        df_concatenated.to_excel(os.path.join(output_dir, f'{tumor_type_name}_coefficients.xlsx'), index=False)
        print(f'DataFrame exported to: {os.path.join(output_dir,  f'{tumor_type_name}_coefficients.xlsx')}')

    return (df_concatenated)

In [1]:
arms_cross_val_coeffs = ml_log_reg_coefficients(ARMS_training_df.iloc[:, 0:20],tumor_type_name='ARSM_try2', l1_ratio=0.1, C=10,  r=5, export=True)

NameError: name 'ml_log_reg_coefficients' is not defined

In [83]:
arms_cross_val_coeffs

,MSH6,PCLAF,UTP18,SEC16A,IPO7,EIF3L,RPAP3,INTS3,BIN1,PPA1,...,CCT2,DARS1,CUL4B,DYNC1I2,AMPD2,Intercept,F1_1,F1_0,F1_weighted,MCC_score
0,9.615715,-1.839995,-3.198187,-0.306665,8.599230,9.738020,5.993231,-1.196944,8.080869,-11.146211,...,21.404736,-0.370488,-20.370987,-10.858083,-20.807662,-2.738711,0.468085,0.951267,0.932285,0.526458
1,9.824755,0.929299,-3.311602,0.680432,4.857691,12.139085,0.158227,-4.493460,11.014196,-13.728125,...,18.570183,5.994456,-20.411627,-8.943347,-21.079502,-4.132075,0.533333,0.973585,0.956289,0.530201
2,4.917284,1.695644,-0.665381,-0.393148,5.978527,8.552324,0.694903,2.124191,9.798453,-9.781009,...,21.048918,2.002228,-18.670322,-10.819759,-22.071330,-5.627919,0.423077,0.940945,0.920600,0.488233
3,10.084075,-0.777086,-2.960253,1.405749,3.169249,9.235836,8.065338,3.381389,6.462130,-13.245694,...,16.387717,5.255697,-18.032231,-9.571617,-24.924898,-7.623822,0.529412,0.969466,0.953693,0.559713
4,10.545113,0.612057,-1.040166,1.845243,3.916952,11.612575,-1.537141,-5.066796,10.134529,-13.285530,...,17.502148,6.762945,-19.672404,-7.003536,-21.610322,-2.366230,0.434783,0.949416,0.929199,0.479433
5,7.616818,0.875121,-5.075690,-0.764358,5.445387,10.253603,3.759117,1.227163,7.224825,-9.088463,...,22.663820,1.824006,-19.741743,-11.386836,-24.068869,-4.756648,0.407407,0.936759,0.915963,0.474745
6,8.664921,-1.350247,-2.019869,-1.057784,6.366013,6.881470,8.214372,4.665654,8.951950,-12.054049,...,18.784007,3.034025,-21.349591,-11.819911,-20.362201,-6.620540,0.526316,0.965517,0.948263,0.556732
7,6.488045,-0.631846,-2.730946,1.649950,7.275765,9.645799,4.511992,-0.645149,9.192600,-13.261562,...,18.989606,3.235259,-17.306606,-8.674287,-22.564882,-7.397456,0.434783,0.949219,0.930780,0.500929
8,8.268101,-0.431480,2.121683,1.828401,3.998965,8.371287,-1.307800,-2.469163,10.828009,-12.326770,...,16.653239,5.817614,-20.022218,-10.183000,-18.648611,-1.434023,0.514286,0.967619,0.949810,0.529104
9,7.587934,-0.253879,-3.821617,2.628337,6.868320,10.905783,3.003761,0.963839,9.707970,-13.500058,...,19.546238,1.905996,-19.719816,-11.728150,-23.304820,-3.220827,0.588235,0.987109,0.971439,0.604829


In [ ]:
#cross validation for final model
def nested_cross_validation_logistic_regression(train_df:pd.DataFrame,classified_by:str, random_state=93):
    """
    classification_criteria: insert the 
    """
    y = train_df['Classifier']  # True values (dependent variable)
    X = train_df.drop(columns=['Sample name', classified_by, 'Classifier'], inplace=True)  # Independent variables (proteins)

    # Define the hyperparameter grid for Logistic Regression
    param_grid = {'C': [0.1, 1, 10]}

    # Set up the Logistic Regression model with L2 regularization (Ridge)
    logreg = LogisticRegression(penalty='elasticnet',
                                solver='saga',
                                l1_ratio=0,  # Ridge regularization
                                max_iter=10000,
                                class_weight='balanced',
                                warm_start=True)

    # Set up MCC scorer
    mcc_scorer = make_scorer(matthews_corrcoef)

    # Set up inner and outer cross-validation
    inner_cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=random_state)
    outer_cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=random_state)

    # Lists to store results
    outer_scores = []
    best_params = []
    inner_fold_cycle = 1
    # Perform nested cross-validation
    for train_idx, test_idx in outer_cv.split(X, y):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        # Inner cross-validation with GridSearchCV
        grid_search = GridSearchCV(estimator=logreg, param_grid=param_grid, cv=inner_cv, scoring=mcc_scorer)
        grid_search.fit(X_train, y_train)

        # Get the best parameters and the score for the inner CV
        best_param = grid_search.best_params_
        best_score = grid_search.best_score_

        # Evaluate the best model on the outer test set
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test)
        outer_score = matthews_corrcoef(y_test, y_pred)

        # Store the results
        outer_scores.append(outer_score)
        best_params.append(best_param)

        # Print the best parameters and the cross-validation score for each fold
        print(f"{inner_fold_cycle} Inner fold best parameter={best_param}, Score={best_score:.4f}, Outer MCC Score(held-out): {outer_score:.4f}")
        inner_fold_cycle += 1
 
    # Print overall mean MCC score
    print(f"Average MCC across all outer folds: {np.mean(outer_scores):.4f}")
    
    return outer_scores, best_params
